In [57]:
import numpy as np
from enum import Enum

# Experiment setup

In [98]:
def generateWorld(height, width):
    world = np.random.randint(1, 10, size = (height, width)) * 1.0
    world[-1][-1] = -10.
    print("World:\n{}".format(world))
    return world

In [99]:
height, width = 5, 5

world = generateWorld(height,width)


World:
[[  8.   3.   9.   4.   2.]
 [  7.   8.   1.   7.   2.]
 [  2.   5.   4.   3.   7.]
 [  7.   7.   2.   9.   9.]
 [  6.   5.   1.   9. -10.]]


# Value iteration and Policy extraction

In [100]:
def valueIteration(current_values, world, iterations = 1, gamma = 0.8):

    for _ in range(iterations):
        old_values = np.copy(current_values)
        
        for r, row in enumerate(world):
            for c, value in enumerate(row):
                best_value = float("inf")
                
                if c < len(row) - 1:
                    current_values[r][c] = min(world[r][c + 1] + gamma * old_values[r][c + 1], best_value)
                    
                if r < len(world) - 1:
                    current_values[r][c] = min(world[r + 1][c] + gamma * old_values[r + 1][c], best_value)

def policyExtraction(current_values, world, gamma = 0.8):
    policy = [[None for _ in row] for row in world]
    
    for r in range(len(world)):
        for c in range(len(world[0])):
            best_value = float("inf")
            
            if c < len(world[0]) - 1:
                next_value = world[r][c + 1] + gamma * current_values[r][c + 1] 
                if next_value < best_value:
                    best_value = next_value
                    policy[r][c] = "R"

            if r < len(world) - 1:
                next_value = world[r + 1][c] + gamma * current_values[r + 1][c] 
                if next_value < best_value:
                    best_value = next_value
                    policy[r][c] = "D"
    return policy

In [102]:
values = np.zeros((height,width))
for _ in range(100):
    
    if _ % 10 == 0:
        print("Iterations: {}".format(_))
        print("Values: \n{}".format(values))
    valueIteration(values, world, gamma = 0.8)
    
VI_policy = policyExtraction(values, world, gamma = 0.8)

print("Optimal policy:")
for line in VI_policy:
    print(line)

Iterations: 0
Values: 
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Iterations: 10
Values: 
[[ 18.789824  19.77728    6.4016    15.672      8.24    ]
 [ 14.73728   14.7216     6.752     10.84       7.8     ]
 [ 15.9216    12.152      3.44       9.8        1.      ]
 [ 11.152      6.44       1.8        1.       -10.      ]
 [  6.44       1.8        1.       -10.         0.      ]]
Iterations: 20
Values: 
[[ 18.789824  19.77728    6.4016    15.672      8.24    ]
 [ 14.73728   14.7216     6.752     10.84       7.8     ]
 [ 15.9216    12.152      3.44       9.8        1.      ]
 [ 11.152      6.44       1.8        1.       -10.      ]
 [  6.44       1.8        1.       -10.         0.      ]]
Iterations: 30
Values: 
[[ 18.789824  19.77728    6.4016    15.672      8.24    ]
 [ 14.73728   14.7216     6.752     10.84       7.8     ]
 [ 15.9216    12.152      3.44       9.8        1.      ]
 [ 11.152      6.44       1.8        1.       -10.      ]


# Policy Iteration

In [114]:
def policyIteration(current_policy, world, iterations = 1, gamma = 0.8):
    
    for _ in range(iterations):
        coeff_mat = np.zeros((world.size - 1, world.size - 1))
        dependent_mat = np.zeros(world.size - 1)
        num_cols = len(world[0])

        for ind in range(len(coeff_mat)):
            r, c = ind % num_cols, (ind - ind %num_cols) //5

            if current_policy[r][c] == "D":
                next_r, next_c = r + 1, c

            else:
                next_r, next_c = r, c + 1

            coeff_mat[ind][num_cols*r + c] = 1

            if num_cols * next_r + next_c == world.size - 1:
                dependent_mat[ind] = world[next_r][next_c]

            else:
                coeff_mat[ind][num_cols*next_r + c] = -gamma
                dependent_mat[ind] = world[next_r][next_c]

            values = np.linalg.lstsq(coeff_mat, dependent_mat, rcond=None)[0]

        values = np.append(values, 0)
        values = np.reshape(values, (-1, num_cols))

        current_policy = policyExtraction(values, world, gamma)
    
    return current_policy,values
    
    

In [115]:
PI_policy = np.array([["R" for _ in row] for row in world])
PI_policy[:,-1] = np.array(["D" for row in world]).reshape(-1)
PI_policy[-1][-1] = None

for _ in range(100):
    if _ % 10 == 0:
        print("Iterations: {}".format(_))
        for line in PI_policy:
            print(line)
    PI_policy, PI_values = policyIteration(PI_policy, world, iterations = 1)

Iterations: 0
['R' 'R' 'R' 'R' 'D']
['R' 'R' 'R' 'R' 'D']
['R' 'R' 'R' 'R' 'D']
['R' 'R' 'R' 'R' 'D']
['R' 'R' 'R' 'R' 'N']
Iterations: 10
['D', 'D', 'D', 'R', 'D']
['D', 'D', 'D', 'D', 'D']
['R', 'R', 'D', 'D', 'D']
['D', 'R', 'D', 'R', 'D']
['R', 'R', 'R', 'R', None]
Iterations: 20
['D', 'D', 'D', 'R', 'D']
['D', 'D', 'D', 'D', 'D']
['R', 'R', 'D', 'D', 'D']
['D', 'R', 'D', 'R', 'D']
['R', 'R', 'R', 'R', None]
Iterations: 30
['D', 'D', 'D', 'R', 'D']
['D', 'D', 'D', 'D', 'D']
['R', 'R', 'D', 'D', 'D']
['D', 'R', 'D', 'R', 'D']
['R', 'R', 'R', 'R', None]
Iterations: 40
['D', 'D', 'D', 'R', 'D']
['D', 'D', 'D', 'D', 'D']
['R', 'R', 'D', 'D', 'D']
['D', 'R', 'D', 'R', 'D']
['R', 'R', 'R', 'R', None]
Iterations: 50
['D', 'D', 'D', 'R', 'D']
['D', 'D', 'D', 'D', 'D']
['R', 'R', 'D', 'D', 'D']
['D', 'R', 'D', 'R', 'D']
['R', 'R', 'R', 'R', None]
Iterations: 60
['D', 'D', 'D', 'R', 'D']
['D', 'D', 'D', 'D', 'D']
['R', 'R', 'D', 'D', 'D']
['D', 'R', 'D', 'R', 'D']
['R', 'R', 'R', 'R', None]


# Q Learning

In [112]:
VI_policy

[['D', 'R', 'D', 'R', 'D'],
 ['D', 'R', 'D', 'R', 'D'],
 ['R', 'R', 'D', 'R', 'D'],
 ['D', 'R', 'D', 'R', 'D'],
 ['R', 'R', 'R', 'R', None]]

In [117]:
for line in PI_values:
    print(line)

[ 4.6    8.8    1.384 -2.5    8.24 ]
[-3.    1.    0.48  3.    7.8 ]
[-6.25000000e+00 -5.00000000e+00 -4.40000000e+00  1.77635684e-15
  1.00000000e+00]
[  1.    -2.5   -8.   -11.25 -10.  ]
[ -6.25  -1.25 -11.25 -10.     0.  ]


In [118]:
values

array([[ 18.789824,  19.77728 ,   6.4016  ,  15.672   ,   8.24    ],
       [ 14.73728 ,  14.7216  ,   6.752   ,  10.84    ,   7.8     ],
       [ 15.9216  ,  12.152   ,   3.44    ,   9.8     ,   1.      ],
       [ 11.152   ,   6.44    ,   1.8     ,   1.      , -10.      ],
       [  6.44    ,   1.8     ,   1.      , -10.      ,   0.      ]])